# 第5回講義 宿題

## 課題

今Lessonで学んだことを元に、MNISTのファッション版 (Fashion MNIST、クラス数10) を多層パーセプトロンによって分類してみましょう。

Fashion MNISTの詳細については以下のリンクを参考にしてください。

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値

Accuracy: 85%

### ルール

- **下のセルで指定されている`x_train`、`y_train`以外の学習データは使わないでください。**
- MLPの実装には`tf`の低レベルAPIのみを用いてください。具体的に以下のモジュールは使用しないでください。
```python
tf.app,
tf.compat,
tf.contrib,
tf.estimator,
tf.gfile,
tf.graph_util,
tf.image,
tf.initializers,
tf.keras,
tf.layers,
tf.logging,
tf.losses,
tf.metrics,
tf.python_io,
tf.resource_loader,
tf.saved_model,
tf.sets,
tf.summary,
tf.sysconfig,
tf.test,
tf.train
```

### 提出方法

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap05`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap05 (code)`を選択して**提出してください。
      - セルに書いたコードを.py形式で保存するためには%%writefileコマンドなどを利用してください（writefileコマンドではファイルの保存のみが行われセル内のpythonコード自体は実行されません。そのため、実際にコードを走らせる際にはwritefileコマンドをコメントアウトしてください）。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法

- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します.
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します.
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します. これを最終的な評価とします.

### データの読み込み
- この部分は修正しないでください。

In [17]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

try:
    del [
        tf.app,
        tf.compat,
        tf.contrib,
        tf.estimator,
        tf.gfile,
        tf.graph_util,
        tf.image,
        tf.initializers,
        tf.keras,
        tf.layers,
        tf.logging,
        tf.losses,
        tf.metrics,
        tf.python_io,
        tf.resource_loader,
        tf.saved_model,
        tf.sets,
        tf.summary,
        tf.sysconfig,
        tf.test,
        tf.train
    ]
except AttributeError:
    print('Unrequired modules are already deleted (Skipped).')

def load_fashionmnist():
    # 学習データ
    x_train = np.load('/root/userspace/public/chap05/data/x_train.npy')
    y_train = np.load('/root/userspace/public/chap05/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('/root/userspace/public/chap05/data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

Unrequired modules are already deleted (Skipped).


### 多層パーセプトロンの実装

全結合層とDropout層からなるMLPを実装します。

順伝播の式を示します。
$$
\begin{align*}
    {\bf u}^{(1)} &= {\bf W}^{(1)\mathrm{T}} {\bf x} + {\bf b}^{(1)} \\
    {\bf h}^{(1)} &= \mathrm{ReLU}({\bf u}^{(1)}) \\
    {\bf \tilde{h}}^{(1)} &= \mathrm{Dropout}({\bf h}^{(1)}) \\
    {\bf u}^{(2)} &= {\bf W}^{(2)\mathrm{T}} {\bf \tilde{h}}^{(1)} + {\bf b}^{(2)} \\
    {\bf h}^{(2)} &= \mathrm{ReLU}({\bf u}^{(2)}) \\
    {\bf \tilde{h}}^{(2)} &= \mathrm{Dropout}({\bf h}^{(2)}) \\
    {\bf u}^{(3)} &= {\bf W}^{(3)\mathrm{T}} {\bf \tilde{h}}^{(2)} + {\bf b}^{(3)} \\
    {\bf y} &= \mathrm{softmax} ({\bf u}^{(3)})
\end{align*}
$$

In [24]:
import tensorboard as tb
tb.show_graph(sess.graph())

AttributeError: module 'tensorboard' has no attribute 'show_graph'

In [27]:
# %%writefile /root/userspace/chap05/submission/submission_code_result3.py

import math
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

tf.reset_default_graph() # グラフのリセット

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

x_train, t_train, x_test = load_fashionmnist()

x_train, x_valid, t_train, t_valid = train_test_split(x_train, t_train, test_size=1000)

# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
#                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# print(tf.shape(x_train))
# plt.figure()
# plt.imshow(x_train[0])
# plt.colorbar()
# plt.grid(False)
# plt.show()
x_train = (x_train.reshape(-1, 784) / 255).astype(np.float32)
x_valid = (x_valid.reshape(-1, 784) / 255).astype(np.float32)


print(tf.shape(t_train))
# t_train = np.eye(10)[t_train].astype(np.float32)
# t_valid = np.eye(10)[t_valid].astype(np.float32)



# 以下Model記述に専念する部分

x = tf.placeholder(tf.float32, [None, 784]) #入力データ
t = tf.placeholder(tf.float32, [None, 10]) #出力データ
is_training = tf.placeholder(tf.bool) #訓練中の信号

Time = tf.Variable(1.0, name ='Time')

class Dense:
    def __init__(self, in_dim, out_dim, function = lambda x: x):
        self.W = tf.Variable(tf.random_uniform(shape = [in_dim, out_dim], minval = -0.08, maxval = 0.08), name = 'W')
        self.b = tf.Variable(tf.zeros(shape = out_dim), name = 'b')
        
        self.function = function
        self.params = [self.W, self.b]
    
    def __call__(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)

class Dropout:
    def __init__(self, dropout_keep_prob = 1.0):
        self.dropout_keep_prob = dropout_keep_prob
        self.params = []
        
    def __call__(self, h):
        return tf.cond(
            pred = is_training, #訓練時に限定する
            # lambda式の引数は今回は指定しないということっぽいな?
            true_fn = lambda : tf.nn.dropout(h, keep_prob = self.dropout_keep_prob),
            false_fn = lambda : h
        )

# tf.log(0)によるnanを防ぐ
def tf_log(x):
    return tf.log(tf.clip_by_value(x, 1e-10, x))    

def compute_l1_reg(params):
    l1_reg = 0
    for param in params:
        l1_reg += tf.reduce_sum(tf.abs(param))
    return l1_reg

def compute_l2_reg(params):
    l2_reg = 0
    for param in params:
        shape_param = tf.shape(param)
        l2_reg += tf.reduce_sum(tf.pow(param,tf.fill(shape_param,2.0)))
    return l2_reg

def sgd(cost, params, eps=0.1):
    grads = tf.gradients(cost, params)
    updates = []
    for grad, param in zip(grads, params):
        updates.append(param.assign_sub(eps*grad))
    return updates

def rmsprop(cost, params, eta=0.001, rho=0.9, eps=1e-7):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        G = tf.Variable(tf.zeros_like(param, dtype = tf.float32), name = 'G')
        updates.append(G.assign(rho * G + (1 - rho) * (grad**2)))
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub((eta/tf.sqrt(G + eps))*grad))
    return updates

# def rmsprop(cost, params, eta=0.001, rho=0.9, eps=1e-7):
#     grads = tf.gradients(cost, params)
#     updates = []
#     for param, grad in zip(params, grads):
#         G = tf.Variable(tf.zeros_like(param, dtype=tf.float32), name='G')
#         updates.append(G.assign(rho * G + (1 - rho) * grad**2))
#         with tf.control_dependencies(updates):
#             updates.append(param.assign_sub(eta / tf.sqrt(G + eps) * grad))
#     return updates

def momentum(cost, params, eta = 0.01, gamma = 0.9):
    grads = tf.gradients(cost, params)
    updates = []
    for grad, param in zip(grads, params):
        v = tf.Variable(tf.zeros_like(param, dtype = tf.float32), name = 'v')
        updates.append(v.assign(eta*grad + gamma*v))
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub(v))
    return updates


def adagrad(cost, params, eta=0.01, eps=1e-7):
    grads = tf.gradients(cost, params)
    updates = []
    for param, grad in zip(params, grads):
        G = tf.Variable(tf.zeros_like(param, dtype=tf.float32), name='G')
        updates.append(G.assign_add(grad**2))
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub(eta / tf.sqrt(G + eps) * grad))
    return updates

# 自分で書いてみたAdam
# notationはGoodfellow参照した
def Adam(cost, params, t, eps = 0.001, rho_1 = 0.9, rho_2 = 0.999, delta = 1e-8):
    grads = tf.gradients(cost, params)
    updates = []
    for grad, param in zip(grads, params):
        s = tf.Variable(tf.zeros_like(param, dtype = tf.float32), name = 's') # バイアス付１次モーメント初期化
        r = tf.Variable(tf.zeros_like(param, dtype = tf.float32), name = 'r') # バイアス付2次モーメント初期化
        updates.append(s.assign(rho_1 * s + (1 - rho_1) * grad))
        updates.append(r.assign(rho_2 * r + (1 - rho_2) * (grad**2)))
        s_hat = s/(1 - rho_1 ** t)
        r_hat = r/(1 - rho_2 ** t)
        with tf.control_dependencies(updates):
            updates.append(param.assign_sub((eps * s_hat)/tf.sqrt(r_hat + delta)))
    return updates



eta = 0.01 # 学習率
dropout_keep_prob = 0.5 # Dropout率
lmd = 0.001 # L2正則化項の係数
batch_size = 100 # バッチサイズ
n_epochs = 200 # epoch数



layers = [
    Dense(784, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 200, tf.nn.relu),
    Dropout(dropout_keep_prob),
    Dense(200, 10, tf.nn.softmax)
]

def get_params(layers):
    params_all = []
    for layer in layers:
        params = layer.params
        params_all.extend(params)
    return params_all

def f_props(layers, h):
    for layer in layers:
        h = layer(h)
    return h

y = f_props(layers, x)
params_all = get_params(layers)
# l2_reg = compute_l2_reg(params_all)
# l1_reg = compute_l1_reg(params_all)
cost = - tf.reduce_mean(tf.reduce_sum(t * tf_log(y), axis=1)) #+ lmd * l1_reg

# updates = rmsprop(cost, params_all, eta)
updates = Adam(cost=cost, params = params_all, t = Time)
train = tf.group(*updates)

n_batches = math.ceil(len(x_train) / batch_size)








with tf.Session(config = config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        x_train, t_train = shuffle(x_train, t_train)
        Time = 1
        for i in range(n_batches):
            Time = Time + 1
            start = i * batch_size
            end = start + batch_size
            sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end], is_training: True})
        y_pred, cost_valid_ = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid, is_training: False})
        if ((epoch+1)%10 == 0):
            print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
                epoch + 1,
                cost_valid_,
                accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
            ))
    y_pred = sess.run(y, feed_dict = {x: x_test, is_training: False})
    y_pred = y_pred.argmax(axis = 1)
    submission = pd.Series(y_pred, name='label')
    submission.to_csv('/root/userspace/chap05/submission/submission_pred_result3.csv', header=True, index_label='id')

Writing /root/userspace/chap05/submission/submission_code_result3.py


#### Results 1:
Adam is used as optimization algorithm.  
784 -> 128 -> D -> 128 -> D -> 10  
```python
eta = 0.01 # 学習率
dropout_keep_prob = 0.5 # Dropout率
lmd = 0.001 # L2正則化項の係数
batch_size = 100 # バッチサイズ
n_epochs = 100 # epoch数

```
EPOCH: 10, Valid Cost: 0.593, Valid Accuracy: 0.783  
EPOCH: 20, Valid Cost: 0.502, Valid Accuracy: 0.823  
EPOCH: 30, Valid Cost: 0.461, Valid Accuracy: 0.848  
EPOCH: 40, Valid Cost: 0.433, Valid Accuracy: 0.858  
EPOCH: 50, Valid Cost: 0.416, Valid Accuracy: 0.860  
EPOCH: 60, Valid Cost: 0.401, Valid Accuracy: 0.862  
EPOCH: 70, Valid Cost: 0.386, Valid Accuracy: 0.870  
EPOCH: 80, Valid Cost: 0.378, Valid Accuracy: 0.870  
EPOCH: 90, Valid Cost: 0.369, Valid Accuracy: 0.870  
EPOCH: 100, Valid Cost: 0.372, Valid Accuracy: 0.875  


#### Results 2:
Adam is used as optimization algorithm.  
784 -> 200 -> D -> 200 -> D -> 10  
```python
eta = 0.01 # 学習率
dropout_keep_prob = 0.5 # Dropout率
lmd = 0.001 # L2正則化項の係数
batch_size = 100 # バッチサイズ
n_epochs = 100 # epoch数

```
EPOCH: 10, Valid Cost: 0.564, Valid Accuracy: 0.791  
EPOCH: 20, Valid Cost: 0.481, Valid Accuracy: 0.819  
EPOCH: 30, Valid Cost: 0.444, Valid Accuracy: 0.828  
EPOCH: 40, Valid Cost: 0.413, Valid Accuracy: 0.846  
EPOCH: 50, Valid Cost: 0.395, Valid Accuracy: 0.853  
EPOCH: 60, Valid Cost: 0.381, Valid Accuracy: 0.861  
EPOCH: 70, Valid Cost: 0.373, Valid Accuracy: 0.868  
EPOCH: 80, Valid Cost: 0.357, Valid Accuracy: 0.877  
EPOCH: 90, Valid Cost: 0.355, Valid Accuracy: 0.879  
EPOCH: 100, Valid Cost: 0.351, Valid Accuracy: 0.878  

#### Results 3:
Adam is used as optimization algorithm.  
784 -> 200 -> D -> 200 -> D -> 200 -> D -> 10  
```python
eta = 0.01 # 学習率
dropout_keep_prob = 0.5 # Dropout率
lmd = 0.001 # L2正則化項の係数
batch_size = 100 # バッチサイズ
n_epochs = 200 # epoch数

```
EPOCH: 10, Valid Cost: 0.546, Valid Accuracy: 0.790  
EPOCH: 20, Valid Cost: 0.465, Valid Accuracy: 0.823  
EPOCH: 30, Valid Cost: 0.427, Valid Accuracy: 0.831  
EPOCH: 40, Valid Cost: 0.406, Valid Accuracy: 0.838  
EPOCH: 50, Valid Cost: 0.387, Valid Accuracy: 0.852  
EPOCH: 60, Valid Cost: 0.364, Valid Accuracy: 0.862  
EPOCH: 70, Valid Cost: 0.360, Valid Accuracy: 0.872  
EPOCH: 80, Valid Cost: 0.352, Valid Accuracy: 0.872  
EPOCH: 90, Valid Cost: 0.347, Valid Accuracy: 0.874  
EPOCH: 100, Valid Cost: 0.352, Valid Accuracy: 0.867  
EPOCH: 110, Valid Cost: 0.338, Valid Accuracy: 0.878  
EPOCH: 120, Valid Cost: 0.345, Valid Accuracy: 0.871  
EPOCH: 130, Valid Cost: 0.332, Valid Accuracy: 0.875  
EPOCH: 140, Valid Cost: 0.340, Valid Accuracy: 0.879  
EPOCH: 150, Valid Cost: 0.327, Valid Accuracy: 0.887  
EPOCH: 160, Valid Cost: 0.333, Valid Accuracy: 0.876  
EPOCH: 170, Valid Cost: 0.331, Valid Accuracy: 0.880  
EPOCH: 180, Valid Cost: 0.333, Valid Accuracy: 0.886  
EPOCH: 190, Valid Cost: 0.339, Valid Accuracy: 0.889  
EPOCH: 200, Valid Cost: 0.342, Valid Accuracy: 0.878  

In [ ]:
# y_pred = sess.run([y], feed_dict = {x: x_test, is_training: False})
# submission = pd.Series(y_pred, name='label')
# submission.to_csv('/root/userspace/chap05/submission/submission_pred.csv', header=True, index_label='id')

In [26]:
! nvidia-smi
tb.show_graph(sess.graph) 
# np.shape(x_train)
# x_train_img = x_train.reshape(-1,28,28)
# plt.figure()
# plt.imshow(x_train_img[0])
# plt.colorbar()
# plt.grid(False)
# plt.show()
# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
#                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(x_train_img[i], cmap=plt.cm.binary)
#     plt.xlabel(class_names[t_train[i,:].argmax()])
# plt.show()


Thu May 23 16:53:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla M60           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   45C    P0    37W / 150W |    250MiB /  7618MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

AttributeError: module 'tensorboard' has no attribute 'show_graph'

In [ ]:
# 今までの実行によりデフォルトグラフ上に溜まったオペレーション
# tf.get_default_graph().get_operations()
# 今までの実行によりデフォルトグラフ上に溜まったVariables
# tf.global_variables()

In [9]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        x_train, t_train = shuffle(x_train, t_train)
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            sess.run(train, feed_dict={x: x_train[start:end], t: t_train[start:end], is_training: True})
        y_pred, cost_valid_ = sess.run([y, cost], feed_dict={x: x_valid, t: t_valid, is_training: False})
        print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
            epoch + 1,
            cost_valid_,
            accuracy_score(t_valid.argmax(axis=1), y_pred.argmax(axis=1))
        ))

EPOCH: 1, Valid Cost: 2.717, Valid Accuracy: 0.181
EPOCH: 2, Valid Cost: 2.708, Valid Accuracy: 0.181
EPOCH: 3, Valid Cost: 2.698, Valid Accuracy: 0.085
EPOCH: 4, Valid Cost: 2.689, Valid Accuracy: 0.085
EPOCH: 5, Valid Cost: 2.680, Valid Accuracy: 0.085
EPOCH: 6, Valid Cost: 2.671, Valid Accuracy: 0.085
EPOCH: 7, Valid Cost: 2.663, Valid Accuracy: 0.085
EPOCH: 8, Valid Cost: 2.654, Valid Accuracy: 0.085
EPOCH: 9, Valid Cost: 2.646, Valid Accuracy: 0.086
EPOCH: 10, Valid Cost: 2.638, Valid Accuracy: 0.124


AttributeError: module 'tensorboard' has no attribute 'show_graph'

In [5]:
if 'session' in locals() and session is not None:
    print('Close interactive session')
    session.close()

In [ ]:

submission = pd.Series(y_pred, name='label')
submission.to_csv('/root/userspace/chap05/submission/submission_pred.csv', header=True, index_label='id')